In [ ]:
from abc import ABC, abstractstaticmethod

import numpy as np

import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Dropout, Flatten, GRU
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.regularizers import l2

import keras_tuner as kt

from sklearn.utils import class_weight

import pickle

from matplotlib import pyplot
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

<h2>
Utils
</h2>

In [ ]:
class HPConfiguration(ABC):
    
    @abstractstaticmethod
    def build_hp_model(hp):
        pass
    
    @abstractstaticmethod
    def get_tuner():
        pass
    
    @abstractstaticmethod
    def get_callbacks():
        pass
    
    @abstractstaticmethod
    def get_tuner_callbacks():
        pass

In [ ]:
basic_classes = [
    'yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go'
]

additional_classes = [
    'zero', 'wow', 'two', 'tree', 'three', 'six', 'sheila', 'seven', 'one', 'nine', 'marvin', 'house', 'happy', 'four', 'five', 'eight', 'dog', 'cat', 'bird', 'bed'
]

all_classes = basic_classes + additional_classes

class_name_to_idx = {class_name: idx for idx, class_name in enumerate(all_classes)}
idx_to_class_name = {idx: class_name for idx, class_name in enumerate(all_classes)}

In [ ]:
def load_datasets(type, full_class=True, augmentation='none'):
    if type == 'mfcc':
        if augmentation == 'none':
            train_x = np.load("numpy_arrays/X_train_mfcc_unaugmented.npy")
            train_y = np.load("numpy_arrays/y_train_unaugmented.npy")
        elif augmentation == 'noice':
            train_x = np.load(
                "numpy_arrays/X_train_mfcc_noice_augmentation.npy")
            train_y = np.load(
                "numpy_arrays/y_train_mfcc_noice_augmentation.npy")
        elif augmentation == 'pitch':
            train_x = np.load(
                "numpy_arrays/X_train_mfcc_pitch_augmentation.npy")
            train_y = np.load(
                "numpy_arrays/y_train_mfcc_pitch_augmentation.npy")
        validation_x = np.load("numpy_arrays/X_validation_mfcc.npy")
        validation_y = np.load("numpy_arrays/y_validation_mfcc.npy")
        test_x = np.load("numpy_arrays/X_test_mfcc.npy")
        test_y = np.load("numpy_arrays/y_test_mfcc.npy")
    elif type == 'mel':
        if augmentation == 'none':
            train_x = np.load(
                "numpy_arrays/X_train_mel_spectogram_unaugmented.npy")
            train_y = np.load(
                "numpy_arrays/y_train_mel_spectogram_unaugmented.npy")
        elif augmentation == 'noice':
            train_x = np.load(
                "numpy_arrays/X_train_mel_spectogram_noice_augmentation.npy")
            train_y = np.load(
                "numpy_arrays/y_train_mel_spectogram_noice_augmentation.npy")
        elif augmentation == 'pitch':
            train_x = np.load(
                "numpy_arrays/X_train_mel_spectogram_pitch_augmentation.npy")
            train_y = np.load(
                "numpy_arrays/y_train_mel_spectogram_pitch_augmentation.npy")
        validation_x = np.load("numpy_arrays/X_validation_mel_spectogram.npy")
        validation_y = np.load("numpy_arrays/y_validation_mel_spectogram.npy")
        test_x = np.load("numpy_arrays/X_test_mel_spectogram.npy")
        test_y = np.load("numpy_arrays/y_test_mel_spectogram.npy")

    if full_class != True:
        train_y[train_y > 9] = 10
        validation_y[validation_y > 9] = 10
        test_y[test_y > 9] = 10

    return train_x, train_y, validation_x, validation_y, test_x, test_y

In [ ]:
def drawConfMatrix(model, x_test, y_test, labels):
    predictions = model.predict(x_test, verbose=True)
    predictions = np.argmax(predictions, axis=1)

    confMat = ConfusionMatrixDisplay(confusion_matrix(y_test, predictions, normalize='true'), display_labels=labels)
    fig, ax = pyplot.subplots(figsize=(8,8))
    confMat.plot(ax = ax,  xticks_rotation = 'vertical')

def plot_accuracy(history, model_name):
    plt.plot(history['accuracy'])
    plt.plot(history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.title(model_name)
    plt.show()

def plot_loss(history, model_name):
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.title(model_name)
    plt.show()

def plot_history(history, model_name):
    plot_accuracy(history, model_name)
    plot_loss(history, model_name)

In [ ]:
class_number = 11
input_shape = (25, 32)
#input_shape = (128, 32)

class SimpleLSTMModel(keras.Model, HPConfiguration):
    def __init__(self, lstm_units, dense_units, **kwargs):
        super().__init__(kwargs)

        self.lstm_1 = LSTM(
            lstm_units, return_sequences=False, input_shape=input_shape)
        self.dense_1 = Dense(dense_units, activation='relu')
        self.dense_2 = Dense(class_number, activation='softmax')

    def call(self, inputs):

        x = self.lstm_1(inputs)
        x = self.dense_1(x)
        x = self.dense_2(x)
        return x

    def build_hp_model(hp):
        lstm_units = hp.Int("lstm_units", min_value=32,
                            max_value=256, step=2, sampling="log")
        dense_units = hp.Int("dense_units", min_value=16,
                             max_value=256, step=2, sampling="log")
        model = SimpleLSTMModel(
            lstm_units=lstm_units, dense_units=dense_units
        )
        model.compile(loss='sparse_categorical_crossentropy',
                      optimizer='adam', metrics=['accuracy'])
        return model

    def get_tuner():
        return kt.Hyperband(SimpleLSTMModel.build_hp_model,
                            objective='val_accuracy',
                            # overwrite=True,
                            max_epochs=50,
                            factor=3,
                            directory='tuner/SimpleLSTMModel',
                            project_name='model'
                            )

    def get_callbacks():
        return [
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20),
            #tf.keras.callbacks.ModelCheckpoint(filepath="models/SuperSimpleConvModel", save_best_only=True)
        ]

    def get_tuner_callbacks():
        return [
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20),
        ]

    def get_best_model():
        tuner = SimpleLSTMModel.get_tuner()
        best_parameters = tuner.get_best_hyperparameters(num_trials=1)[0]
        return SimpleLSTMModel.build_hp_model(best_parameters)


class IntermediateLSTMModel(keras.Model, HPConfiguration):
    def __init__(self, lstm_units_1, lstm_units_2, dense_units, **kwargs):
        super().__init__(kwargs)

        self.lstm_1 = LSTM(
            lstm_units_1, return_sequences=True, input_shape=input_shape)
        self.lstm_2 = LSTM(lstm_units_2, return_sequences=False)
        self.dense_1 = Dense(dense_units, activation='relu')
        self.dense_2 = Dense(class_number, activation='softmax')

    def call(self, inputs):
        x = self.lstm_1(inputs)
        x = self.lstm_2(x)
        x = self.dense_1(x)
        x = self.dense_2(x)
        return x

    def build_hp_model(hp):
        lstm_units_1 = hp.Int("lstm_units_1", min_value=32,
                              max_value=256, step=2, sampling="log")
        lstm_units_2 = hp.Int("lstm_units_2", min_value=32,
                              max_value=256, step=2, sampling="log")
        dense_units = hp.Int("dense_units", min_value=16,
                             max_value=256, step=2, sampling="log")
        model = IntermediateLSTMModel(
            lstm_units_1=lstm_units_1, lstm_units_2=lstm_units_2, dense_units=dense_units
        )
        model.compile(loss='sparse_categorical_crossentropy',
                      optimizer='adam', metrics=['accuracy'])
        return model

    def get_tuner():
        return kt.Hyperband(IntermediateLSTMModel.build_hp_model,
                            objective='val_accuracy',
                            # overwrite=True,
                            max_epochs=50,
                            factor=3,
                            directory='tuner/IntermediateLSTMModel',
                            project_name='model'
                            )

    def get_callbacks():
        return [
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20),
            #tf.keras.callbacks.ModelCheckpoint(filepath="models/SuperSimpleConvModel", save_best_only=True)
        ]

    def get_tuner_callbacks():
        return [
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20),
        ]

    def get_best_model():
        tuner = IntermediateLSTMModel.get_tuner()
        best_parameters = tuner.get_best_hyperparameters(num_trials=1)[0]
        return IntermediateLSTMModel.build_hp_model(best_parameters)


class AdvancedLSTMModel(keras.Model, HPConfiguration):

    def __init__(self, lstm_units_1, lstm_units_2, dense_units_1, dense_units_2, **kwargs):
        super().__init__(kwargs)

        self.lstm_1 = LSTM(
            lstm_units_1, return_sequences=True, input_shape=input_shape)
        self.lstm_2 = LSTM(lstm_units_2, return_sequences=False)
        self.dense_1 = Dense(dense_units_1, activation='relu')
        self.dense_2 = Dense(dense_units_2, activation='relu')
        self.dense_3 = Dense(class_number, activation='softmax')

    def call(self, inputs):
        x = self.lstm_1(inputs)
        x = self.lstm_2(x)
        x = self.dense_1(x)
        x = self.dense_2(x)
        x = self.dense_3(x)
        return x

    def build_hp_model(hp):
        lstm_units_1 = hp.Int("lstm_units_1", min_value=32,
                              max_value=256, step=2, sampling="log")
        lstm_units_2 = hp.Int("lstm_units_2", min_value=32,
                              max_value=256, step=2, sampling="log")
        dense_units_1 = hp.Int("dense_units_1", min_value=16,
                               max_value=256, step=2, sampling="log")
        dense_units_2 = hp.Int("dense_units_2", min_value=16,
                               max_value=256, step=2, sampling="log")
        model = AdvancedLSTMModel(
            lstm_units_1=lstm_units_1, lstm_units_2=lstm_units_2, dense_units_1=dense_units_1, dense_units_2=dense_units_2
        )
        model.compile(loss='sparse_categorical_crossentropy',
                      optimizer='adam', metrics=['accuracy'])
        return model

    def get_tuner():
        return kt.Hyperband(AdvancedLSTMModel.build_hp_model,
                            objective='val_accuracy',
                            # overwrite=True,
                            max_epochs=50,
                            factor=3,
                            directory='tuner/AdvancedLSTMModel',
                            project_name='model'
                            )

    def get_callbacks():
        return [
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20),
            #tf.keras.callbacks.ModelCheckpoint(filepath="models/SuperSimpleConvModel", save_best_only=True)
        ]

    def get_tuner_callbacks():
        return [
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20),
        ]

    def get_best_model():
        tuner = AdvancedLSTMModel.get_tuner()
        best_parameters = tuner.get_best_hyperparameters(num_trials=1)[0]
        return AdvancedLSTMModel.build_hp_model(best_parameters)
    
class SimpleGRUModel(keras.Model, HPConfiguration):
    def __init__(self, gru_units, dense_units, **kwargs):
        super().__init__(kwargs)

        self.gru_1 = GRU(
            gru_units, return_sequences=False, input_shape=input_shape)
        self.dense_1 = Dense(dense_units, activation='relu')
        self.dense_2 = Dense(class_number, activation='softmax')

    def call(self, inputs):

        x = self.gru_1(inputs)
        x = self.dense_1(x)
        x = self.dense_2(x)
        return x

    def build_hp_model(hp):
        gru_units = hp.Int("gru_units", min_value=32,
                            max_value=256, step=2, sampling="log")
        dense_units = hp.Int("dense_units", min_value=16,
                             max_value=256, step=2, sampling="log")
        model = SimpleGRUModel(
            gru_units=gru_units, dense_units=dense_units
        )
        model.compile(loss='sparse_categorical_crossentropy',
                      optimizer='adam', metrics=['accuracy'])
        return model

    def get_tuner():
        return kt.Hyperband(SimpleGRUModel.build_hp_model,
                            objective='val_accuracy',
                            # overwrite=True,
                            max_epochs=50,
                            factor=3,
                            directory='tuner/SimpleGRUModel',
                            project_name='model'
                            )

    def get_callbacks():
        return [
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20),
            #tf.keras.callbacks.ModelCheckpoint(filepath="models/SuperSimpleConvModel", save_best_only=True)
        ]

    def get_tuner_callbacks():
        return [
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20),
        ]

    def get_best_model():
        tuner = SimpleGRUModel.get_tuner()
        best_parameters = tuner.get_best_hyperparameters(num_trials=1)[0]
        return SimpleGRUModel.build_hp_model(best_parameters)


<h1>
Hyperparameter tuning
</h1>

In [ ]:
x_train, y_train, x_validation, y_validation, x_test, y_test = load_datasets('mfcc', False)

np.unique(y_train, return_counts=True)

In [ ]:
#model_classes = [SimpleLSTMModel, IntermediateLSTMModel, AdvancedLSTMModel, SimpleGRUModel]
model_classes = [SimpleGRUModel]
# HYPERPARAMETER TUNING
for model_class in model_classes:
    print(model_class)
    tuner = model_class.get_tuner()
    #print(tuner)
    tuner.search(x_train, y_train, batch_size=256, epochs=50, validation_data=(x_validation, y_validation), callbacks=model_class.get_tuner_callbacks())
    best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
for model_class in model_classes:
    tuner = model_class.get_tuner()
    best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
    print(best_hps.values)


<h1>
Class weight influence experiment
</h1>

In [ ]:
n_epochs = 64

early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')

for model_class in [IntermediateLSTMModel]:
    tuner = model_class.get_tuner()
    best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

    for i in range(5):
        keras.backend.clear_session()
        best_model = model_class.build_hp_model(best_hps)
        history = best_model.fit(x_train, y_train,
                                batch_size=32,
                                epochs=n_epochs,
                                validation_data=(x_validation, y_validation),
                                callbacks=[early_stop],
                                verbose=1)
        best_model.save('models/' + model_class.__name__ + '/' + str(i))
        with open(f'./train_history/' + model_class.__name__ + '_' + str(i), 'wb') as file_pi:
            pickle.dump(history.history, file_pi)

In [ ]:
n_epochs = 64

early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')

class_weights = class_weight.compute_class_weight(
    class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(list(enumerate(class_weights)))

for model_class in [IntermediateLSTMModel]:
    tuner = model_class.get_tuner()
    best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

    for i in range(5):
        keras.backend.clear_session()
        best_model = model_class.build_hp_model(best_hps)
        history = best_model.fit(x_train, y_train,
                                batch_size=32,
                                epochs=n_epochs,
                                validation_data=(x_validation, y_validation),
                                callbacks=[early_stop],
                                verbose=1,
                                class_weight=class_weights)
        best_model.save('models/' + model_class.__name__ + '_cw/' + str(i))
        with open(f'./train_history/' + model_class.__name__ + '_cw_' + str(i), 'wb') as file_pi:
            pickle.dump(history.history, file_pi)

<h1>
Different networks on MFCC
</h1>

In [ ]:
n_epochs = 64

early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')

for model_class in [SimpleGRUModel, SimpleLSTMModel, AdvancedLSTMModel]:
    tuner = model_class.get_tuner()
    best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

    for i in range(5):
        keras.backend.clear_session()
        best_model = model_class.build_hp_model(best_hps)
        history = best_model.fit(x_train, y_train,
                                batch_size=32,
                                epochs=n_epochs,
                                validation_data=(x_validation, y_validation),
                                callbacks=[early_stop],
                                verbose=1)
        best_model.save('models/' + model_class.__name__ + '/' + str(i))
        with open(f'./train_history/' + model_class.__name__ + '_' + str(i), 'wb') as file_pi:
            pickle.dump(history.history, file_pi)

<h1>
Augmentations
</h1>

In [ ]:
x_train, y_train, x_validation, y_validation, x_test, y_test = load_datasets('mfcc', False, augmentation='noice')

np.unique(y_train, return_counts=True)

In [ ]:
n_epochs = 64

early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')

for model_class in [IntermediateLSTMModel]:
    tuner = model_class.get_tuner()
    best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

    for i in range(5):
        keras.backend.clear_session()
        best_model = model_class.build_hp_model(best_hps)
        history = best_model.fit(x_train, y_train,
                                batch_size=32,
                                epochs=n_epochs,
                                validation_data=(x_validation, y_validation),
                                callbacks=[early_stop],
                                verbose=1)
        best_model.save('models/' + model_class.__name__ + '_noice/' + str(i))
        with open(f'./train_history/' + model_class.__name__ + '_noice_' + str(i), 'wb') as file_pi:
            pickle.dump(history.history, file_pi)

In [ ]:
x_train, y_train, x_validation, y_validation, x_test, y_test = load_datasets('mfcc', False, augmentation='pitch')

np.unique(y_train, return_counts=True)

In [ ]:
n_epochs = 64

early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')

for model_class in [IntermediateLSTMModel]:
    tuner = model_class.get_tuner()
    best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

    for i in range(5):
        keras.backend.clear_session()
        best_model = model_class.build_hp_model(best_hps)
        history = best_model.fit(x_train, y_train,
                                batch_size=32,
                                epochs=n_epochs,
                                validation_data=(x_validation, y_validation),
                                callbacks=[early_stop],
                                verbose=1)
        best_model.save('models/' + model_class.__name__ + '_pitch/' + str(i))
        with open(f'./train_history/' + model_class.__name__ + '_pitch_' + str(i), 'wb') as file_pi:
            pickle.dump(history.history, file_pi)

<h1>
Dropout & L2 Regularization
</h1>

In [ ]:
from keras.regularizers import l2
from keras.layers import Dropout

x_train, y_train, x_validation, y_validation, x_test, y_test = load_datasets('mfcc', False)

n_epochs = 64

early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')

In [ ]:
def create_dropout_model(dropout, input_shape=(25, 32)):
    model = keras.Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(LSTM(256, return_sequences=False))
    model.add(Dropout(dropout))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(class_number, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model


def create_regularization_model(reg, dropout=0.0, input_shape=(25, 32)):
    model = keras.Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=input_shape, kernel_regularizer=reg))
    if dropout > 0.0:
        model.add(Dropout(dropout))
    model.add(LSTM(256, return_sequences=False, kernel_regularizer=reg))
    if dropout > 0.0:
        model.add(Dropout(dropout))
    model.add(Dense(32, activation='relu', kernel_regularizer=reg))
    if dropout > 0.0:
        model.add(Dropout(dropout))
    model.add(Dense(class_number, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model


In [ ]:
for i in range(5):
    keras.backend.clear_session()
    model =  create_dropout_model(0.2)
    history = model.fit(x_train, y_train,
                            batch_size=32,
                            epochs=n_epochs,
                            validation_data=(x_validation, y_validation),
                            callbacks=[early_stop],
                            verbose=1)
    model.save('models/d02/' + str(i))
    with open('./train_history/d02_' + str(i), 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

for i in range(5):
    keras.backend.clear_session()
    model =  create_dropout_model(0.1)
    history = model.fit(x_train, y_train,
                            batch_size=32,
                            epochs=n_epochs,
                            validation_data=(x_validation, y_validation),
                            callbacks=[early_stop],
                            verbose=1)
    model.save('models/d01/' + str(i))
    with open('./train_history/d01_' + str(i), 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

In [ ]:
for i in range(5):
    keras.backend.clear_session()
    model =  create_regularization_model(l2(0.02))
    history = model.fit(x_train, y_train,
                            batch_size=32,
                            epochs=n_epochs,
                            validation_data=(x_validation, y_validation),
                            callbacks=[early_stop],
                            verbose=1)
    model.save('models/r002/' + str(i))
    with open('./train_history/r002_' + str(i), 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

for i in range(5):
    keras.backend.clear_session()
    model =  create_regularization_model(l2(0.01))
    history = model.fit(x_train, y_train,
                            batch_size=32,
                            epochs=n_epochs,
                            validation_data=(x_validation, y_validation),
                            callbacks=[early_stop],
                            verbose=1)
    model.save('models/r001/' + str(i))
    with open('./train_history/r001_' + str(i), 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

In [ ]:
for i in range(5):
    keras.backend.clear_session()
    model =  create_regularization_model(l2(0.02), 0.2)
    history = model.fit(x_train, y_train,
                            batch_size=32,
                            epochs=n_epochs,
                            validation_data=(x_validation, y_validation),
                            callbacks=[early_stop],
                            verbose=1)
    model.save('models/r002d02/' + str(i))
    with open('./train_history/r002d02_' + str(i), 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

for i in range(5):
    keras.backend.clear_session()
    model =  create_regularization_model(l2(0.01),0.2)
    history = model.fit(x_train, y_train,
                            batch_size=32,
                            epochs=n_epochs,
                            validation_data=(x_validation, y_validation),
                            callbacks=[early_stop],
                            verbose=1)
    model.save('models/r001d02/' + str(i))
    with open('./train_history/r001d02_' + str(i), 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

<h1>
CNN
</h1>

In [ ]:
x_train, y_train, x_validation, y_validation, x_test, y_test = load_datasets('mel', False)

input_shape = (128, 32, 1)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
n_classes = 11
n_epochs = 64

early_stop = EarlyStopping(monitor='val_loss',
                           patience=5,
                           restore_best_weights=True,
                           mode='min')

def create_vgg_model(con_base, n_classes, optimizer, fine_tune=False):

    conv_base = con_base
    
    for layer in conv_base.layers:
        layer.trainable = fine_tune

    top_model = conv_base.output
    top_model = Flatten()(top_model)
    top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dropout(0.5)(top_model)
    top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dropout(0.5)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)

    model = Model(inputs=conv_base.input, outputs=output_layer)

    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
for i in range(1, 5):
    print(i)
    keras.backend.clear_session()
    vgg16 = tf.keras.applications.VGG16(include_top=False,
                                    weights=None,
                                    input_shape=input_shape)
    vgg_model = create_vgg_model(vgg16, n_classes, optimizer, fine_tune=True)
    vgg_history = vgg_model.fit(x_train, y_train,
                            batch_size=32,
                            epochs=n_epochs,
                            validation_data=(x_validation, y_validation),
                            callbacks=[early_stop],
                            verbose=1)
    vgg_model.save('models/vgg16_/' + str(i))
    with open('./train_history/vgg16_' + str(i), 'wb') as file_pi:
        pickle.dump(vgg_history.history, file_pi)

<h1>
Different networks on MEL
</h1>

In [ ]:
x_train, y_train, x_validation, y_validation, x_test, y_test = load_datasets('mel', False)

np.unique(y_train, return_counts=True)

In [ ]:
n_epochs = 64

early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')

#for model_class in [IntermediateLSTMModel, AdvancedLSTMModel]:
for model_class in [AdvancedLSTMModel]:
    tuner = model_class.get_tuner()
    best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

    for i in range(2,5):
        keras.backend.clear_session()
        best_model = model_class.build_hp_model(best_hps)
        history = best_model.fit(x_train, y_train,
                                batch_size=32,
                                epochs=n_epochs,
                                validation_data=(x_validation, y_validation),
                                callbacks=[early_stop],
                                verbose=1)
        best_model.save('models/mel_' + model_class.__name__ + '/' + str(i))
        with open(f'./train_history/mel_' + model_class.__name__ + '_' + str(i), 'wb') as file_pi:
            pickle.dump(history.history, file_pi)

<h1>
Models reader & statistics generator
</h1>

<h3>
Model reader
</h3>


In [ ]:
#x_train, y_train, x_validation, y_validation, x_test, y_test = load_datasets('mfcc', False)
x_train, y_train, x_validation, y_validation, x_test, y_test = load_datasets('mel', False)

In [ ]:
folder_path = 'models/mel_AdvancedLSTMModel/'
accuracies = []

for i in range(5):
    model = keras.models.load_model(folder_path + str(i))
    test_loss, test_accuracy = model.evaluate(x=x_test, y=y_test) 
    accuracies.append(test_accuracy)

avg = np.average(accuracies)
std = np.std(accuracies)
minimal = np.min(accuracies)
maximal = np.max(accuracies)
print(f"model: {folder_path}, avg: {avg}, std: {std}, min: {minimal}, max: {maximal}")
print('==============')

<h3>
History reader
</h3>

In [ ]:
filename = './train_history/mel_AdvancedLSTMModel_2'

with open(filename, "rb") as file_pi:
    history = pickle.load(file_pi)

plot_history(history, 'Advanced LSTM Model on Mel dataset')